# 2D Navier-Stokes equations

Note que las primeras derivadas de una función de dos variables, $x,y$ están dadas por 
$$ \frac{\partial f}{\partial x} = \frac{f(x+h,y)-f(x,y)}{h} = \frac{f_{i+1,j}-f_{i,j}}{h} = \frac{f_{i+1,j}-f_{i-1,j}}{2h} $$
$$ \frac{\partial f}{\partial y} = \frac{f(x,y+h)-f(x,y)}{h} = \frac{f_{i,j+1}-f_{i,j}}{h} = \frac{f_{i,j+1}-f_{i,j-1}}{2h} $$
$$ \frac{\partial^2 f}{\partial x^2} = \frac{f(x-h,y)-2f(x,y)+f(x+h,y}{h^2} = \frac{f_{i-1,j}-2f_{i,j}+f_{i+1,j}}{h^2}$$
$$ \frac{\partial^2 f}{\partial y^2} = \frac{f(x,y-h)-2f(x,y)+f(x,y+h}{h^2} = \frac{f_{i,j-1}-2f_{i,j}+f_{i,j-1}}{h^2}$$

Buscamos resolver las siguientes ecuaciones diferenciales para $u(x,y), w(x,y)$
$$-w = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial ^2 y}$$
$$\nu \left( \frac{\partial^2 w}{\partial x^2} + \frac{\partial^2 w}{\partial ^2 y} \right) =\frac{\partial u}{\partial y} \frac{\partial w}{\partial x} - \frac{\partial u}{\partial x}\frac{\partial w}{\partial y} $$
Utilizando el método de diferencias finitas esto queda igual a 

$$-w_{i,j} = \frac{u_{i-1,j}-2u_{i,j}+u_{i+1,j}}{h^2} + \frac{u_{i,j-1}-2u_{ij}+u_{i,j+1}}{h^2} = \frac{u_{i-1,j}+u_{i+1,j}+u_{i,j-1}+u_{i,j+1}-4u_{i,j}}{h^2} $$

$$\nu \left( \frac{w_{i-1,j}+w_{i+1,j}+w_{i,j-1}+w_{i,j+1}-4w_{i,j}}{h^2} \right) = \frac{u_{i,j+1}-u_{i,j-1}}{2h} \frac{w_{i+1,j}-w_{i-1,j}}{2h}  - \frac{u_{i+1,j}-u_{i-1,j}}{2h}\frac{w_{i,j+1}-w_{i,j-1}}{2h} $$


De esta primera ecuación se obtiene

$$ u_{i,j} = \frac{1}{4} \left(u_{i-1,j}+u_{i+1,j}+u_{i,j-1}+u_{i,j+1} + h^2 w_{i,j} \right) $$
De la segunda se tiene,
$$ w_{i,j} = \frac{1}{4} \left(w_{i-1,j}+w_{i+1,j}+w_{i,j-1}+w_{i,j+1} \right) - \frac{1}{16 \nu} (u_{i,j+1}-u_{i,j-1})(w_{i+1,j}-w_{i-1,j}) +  \frac{1}{16 \nu}(u_{i+1,j}-u_{i-1,j})(w_{i,j+1}-w_{i,j-1}) $$
Donde, en unidades normales se tiene que $R= \frac{1}{\nu} = \frac{V_0 h}{\nu}$